# Seminar_1 

**Exercise 1**

In [28]:
class RGBto_YUV:
    def YUV_to_RGB(Y, U,V):
        B = round(1.163 * (Y -16) + 2.018 * (U - 128))
        G = round(1.164 * (Y - 16) - 0.813 * (V - 128) - 0.391 * (U - 128))
        R = round(1.164 * (Y - 16) + 1.596 * (V - 128)) 
        return(R, G, B)
    
    def RGB_to_YUV(R,G,B):
        Y = round(0.257 * R + 0.504 * G + 0.098 * B +16)
        U = round(-0.148 * R - 0.291 * G + 0.439 * B +128)
        V = round(0.439 * R - 0.368 * G - 0.071 * B +128)
        return(Y,U,V)
        

In [29]:
Y, U, V = 100, 128, 128

# Convertir YUV a RGB
R, G, B = RGBto_YUV.YUV_to_RGB(Y, U, V)
print(f"Valores RGB: R = {R}, G = {G}, B = {B}")

Valores RGB: R = 98, G = 98, B = 98


In [30]:
# Convertir RGB a YUV
Y, U, V = RGBto_YUV.RGB_to_YUV(R, G, B)
print(f"Valores YUV: Y = {Y}, U = {U}, V = {V}")

Valores YUV: Y = 100, U = 128, V = 128


**Exercise 2**

In [31]:
import subprocess

input_image = "/Users/viktoriaolmedo/Downloads/escut-barca.png"
output_image = "/Users/viktoriaolmedo/Downloads/resized_image.png"

#modificar per path de cada ordinador


def resize_image(input_image, output_image, width, height, quality = 28):

    command = ["ffmpeg","-y","-loglevel", "info","-i", input_image,"-vf", f"scale={width}:{height}",output_image ]

    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(result.stdout)
        print(f"Resized image saved to {output_image}")
    except subprocess.CalledProcessError as e:
        print("Error ocurred while running FFmpeg", e.stderr)


#ffmpeg -i input.jpg -vf scale=320:240 output_320x240.png

resize_image(input_image, output_image, 320, 240, quality=28)

Error ocurred while running FFmpeg ffmpeg version 2024-09-26-git-f43916e217-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafoun

**Exercise 3**

We first implement it with a matrix 8x8 in order to check if the algorithm is really working. 

In [32]:
import numpy as np

matrix = np.arange(1, 65).reshape(8, 8)

def zigzag_traversal(matrix):
    result = [None] * 64  
    n = matrix.shape[0]  
    
    a = 0  
    i = 0 
    j = 0 
    result[a] = matrix[i][j]  # First value
    
    while a < 63:  
        #step 1: (one to the right)
        if j == n - 1:  
            i += 1
        else:
            j += 1

        a += 1
        if a < 64:
            result[a] = matrix[i][j]
        
        #step 2: (Diagonal down to the left until reaching j = 0)
        while i + 1 < n and j - 1 >= 0 and a < 63:
            i += 1
            j -= 1
            a += 1
            result[a] = matrix[i][j]
        
        #step 3: (one down) 
        if i == n - 1:  
            j += 1
        else:
            i += 1

        a += 1
        if a < 64:
            result[a] = matrix[i][j]
        
        #step 4: (Diagonal up to the right until reaching i = 0)
        while j + 1 < n and i - 1 >= 0 and a < 63:
            i -= 1
            j += 1
            a += 1
            result[a] = matrix[i][j]
    
    return result

# Obtener el recorrido zig-zag
zigzag_result = zigzag_traversal(matrix)

# Mostrar la matriz y el recorrido zig-zag
print("Matriz 8x8:")
print(matrix)
print("\nRecorrido Zig-Zag:")
print(zigzag_result)


Matriz 8x8:
[[ 1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16]
 [17 18 19 20 21 22 23 24]
 [25 26 27 28 29 30 31 32]
 [33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48]
 [49 50 51 52 53 54 55 56]
 [57 58 59 60 61 62 63 64]]

Recorrido Zig-Zag:
[1, 2, 9, 17, 10, 3, 4, 11, 18, 25, 33, 26, 19, 12, 5, 6, 13, 20, 27, 34, 41, 49, 42, 35, 28, 21, 14, 7, 8, 15, 22, 29, 36, 43, 50, 57, 58, 51, 44, 37, 30, 23, 16, 24, 31, 38, 45, 52, 59, 60, 53, 46, 39, 32, 40, 47, 54, 61, 62, 55, 48, 56, 63, 64]


Then we implemented for images. We use the resize_image function generated in the previous exercise in order to obtain a nxn image. Then we implement the zig zag function and obtain the expected output. First we implemented in Black and White, but then we realize that maybe it was needed to implemented for rgb images, so we added it to the code.

We can try and change the values of the resize image. In order to check if the algorithm is still working for this case with images, we have set the values to 16, because then we can see the complete matrix. 
Then we change it to a bigger value, 420 in this case.  

In [62]:
import numpy as np
import cv2
import subprocess

def load_image_bw(image_path):
    return cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  

def load_image_rgb(image_path):
    return cv2.imread(image_path)  

def serpentine(matrix):
    n = matrix.shape[0]  
    result = [None] * (n * n)  
    
    a = 0  
    i = 0 
    j = 0 
    result[a] = matrix[i, j]  # First value
    
    while a < n * n - 1:  
        #step 1: (one to the right)
        if j == n - 1:  
            i += 1
        else:
            j += 1

        a += 1
        if a < n * n:
            result[a] = matrix[i, j]
        
        #step 2: (Diagonal down to the left until reaching j = 0)
        while i + 1 < n and j - 1 >= 0 and a < n * n - 1:
            i += 1
            j -= 1
            a += 1
            result[a] = matrix[i, j]
        
        #step 3: (one down)
        if i == n - 1:  
            j += 1
        else:
            i += 1

        a += 1
        if a < n * n:
            result[a] = matrix[i, j]
        
        #step 4: (Diagonal up to the right until reaching i = 0)
        while j + 1 < n and i - 1 >= 0 and a < n * n - 1:
            i -= 1
            j += 1
            a += 1
            result[a] = matrix[i, j]
    
    return result


# Inputs and Outputs
image = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/first_seminar/logo_fcb.jpeg"
input_image = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/first_seminar/logo_fcb_resize.jpeg"
resize_image(image, input_image, 420, 420, quality=28)
output_image_bw = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/first_seminar/serpentine_bw.jpeg"
output_image_rgb = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/first_seminar/serpentine_rgb.jpeg"

##################### Black and White case #####################################

#We load the resize image in BW
image_bw = load_image_bw(input_image)

if image_bw is None:
    print("Error loading the BW image")
else:
    # Print the matrix to then check it the convertion is done well. 
    print("Original Matrix in Black and White:")
    print(image_bw)


# Check if it's a nxn matrix
if image_bw is None:
    print("Error loading the BW image")
else:
    assert image_bw.shape[0] == image_bw.shape[1], "The resize BW image is not nxn."

    serpentine_result_bw = serpentine(image_bw)

    print("Serpentine of image in Black and White:")
    print(serpentine_result_bw)

    n = image_bw.shape[0]
    serpentine_image_bw = np.array(serpentine_result_bw).reshape((n, n))

    # Safe the output
    cv2.imwrite(output_image_bw, serpentine_image_bw)
    print(f"Output in BW safe in: {output_image_bw}")


############################# RGB CASE ############################
def serpentine_color(image):
    channels = cv2.split(image)  # Separate into B, G, and R channels
    serpentine_channels = []

    for idx, channel in enumerate(channels):
        serpentine_result = serpentine(channel)
        
        # Print serpentine result for each channel
        print(f"Serpentine traversal output for channel {idx}:")
        print(serpentine_result)
        
        n = channel.shape[0]
        serpentine_matrix = np.array(serpentine_result).reshape((n, n))
        serpentine_channels.append(serpentine_matrix)

    # Merge channels back into a color image
    serpentine_image_color = cv2.merge(serpentine_channels)
    return serpentine_image_color


#We load the resize image in BW
image_rgb = load_image_rgb(input_image)


if image_rgb is None:
    print("Error loading the image in rgb")
else:
    # Print the matrix to then check it the conversion is done well. 
    print("Original Matrix rgb:")
    print(image_rgb)


if image_rgb is None:
    print("Error loading the image")
else:
    # Check image is nxn
    assert image_rgb.shape[0] == image_rgb.shape[1], "Input image rgb is not nxn."
    
    serpentine_result_rbg = serpentine_color(image_rgb)
    
    print("Output matrix of rgb image",serpentine_result_rbg)
    
    cv2.imwrite(output_image, serpentine_result_rbg)
    print(f"Serpentine image in rgb safe in: {output_image_rgb}")



Resized image saved to /Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/first_seminar/logo_fcb_resize.jpeg
Original Matrix in Black and White:
[[216 216 216 ... 216 216 216]
 [216 216 216 ... 216 216 216]
 [216 216 216 ... 216 216 216]
 ...
 [216 216 216 ... 216 216 216]
 [216 216 216 ... 216 216 216]
 [216 216 216 ... 216 216 216]]
Serpentine of image in Black and White:
[216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 216, 21

**Exercise 4**